# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_fscore_support,accuracy_score,label_ranking_average_precision_score
from sklearn.model_selection  import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
import re

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to /home/jey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# # load data from database
# engine = create_engine('sqlite:///InsertDatabaseName.db')
# df = 
# X = 
# Y = # load data from database
engine = create_engine('sqlite:///DisasterResponseJupter.db')

inspector = inspect(engine)

# Get table information
print(inspector.get_table_names())
df = pd.read_sql("SELECT * FROM Message", engine)
# X = 
# Y = 
df.head()

['Message']


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X_names = ['message']
X = df['message'].values
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [4]:
Y_names = list(set(df.columns.values) - set( ['id','message', 'original', 'genre']))
Y = df[Y_names]
Y

,medical_products,buildings,earthquake,request,storm,aid_related,infrastructure_related,transport,death,floods,...,water,shelter,weather_related,refugees,security,military,other_infrastructure,hospitals,food,child_alone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7,0,0,0,1,0,1,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
category_names = Y.columns.values
category_names

array(['medical_products', 'buildings', 'earthquake', 'request', 'storm',
       'aid_related', 'infrastructure_related', 'transport', 'death',
       'floods', 'aid_centers', 'fire', 'related', 'other_weather',
       'tools', 'direct_report', 'search_and_rescue', 'clothing',
       'medical_help', 'electricity', 'cold', 'shops', 'other_aid',
       'missing_people', 'money', 'offer', 'water', 'shelter',
       'weather_related', 'refugees', 'security', 'military',
       'other_infrastructure', 'hospitals', 'food', 'child_alone'],
      dtype=object)

In [6]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            62G         15G         45G        160K        1.3G         46G
Swap:          975M        319M        656M


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    #init stopwords and WordNetLemmatizer
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
     
    # tokenize text
    tokens = word_tokenize(text)
     
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
 
    return tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:

pipeline =  Pipeline([
#     ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidfvect', TfidfVectorizer(tokenizer=tokenize, stop_words='english')),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
X_train

array(['KRCS plans to support those who have been affected by the floods for four months through emergency response and preparation for further displacement where distribution of seeds and tools to farming communities will take place where needed.',
       'Reports indicate that over 27 million people from 25 districts were affected by the severe cyclonic storms.',
       'Is it common for earthquakes to hit several times in a year or even the same month? Send my response now please so I can know what to do because I have a little emessage ends - could mean a child ',
       ...,
       'A presidential decree mandated a $412 million compensation fund, but the company has insisted on paying 20 percent now and the rest within two years.',
       'I need to where to put the dead bodies i am located in Carrefour. ',
       "I'd like to subscribe to the program. "], dtype=object)

In [10]:
y_train

,medical_products,buildings,earthquake,request,storm,aid_related,infrastructure_related,transport,death,floods,...,water,shelter,weather_related,refugees,security,military,other_infrastructure,hospitals,food,child_alone
23852,0,0,0,0,0,1,0,0,0,1,...,0,0,1,1,0,0,0,0,1,0
13465,0,0,0,0,1,1,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
5753,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
20202,0,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
13761,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
470,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
25750,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22772,0,0,1,0,0,1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
17575,0,0,0,1,0,1,0,1,0,1,...,0,0,1,0,1,0,0,0,0,0
3284,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
pipeline.fit(X_train, y_train)

/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  sorted(inconsistent))
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jey/anaconda3/lib/python3.

/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('tfidfvect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=T...           oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
y_preds = pd.DataFrame(pipeline.predict(X_test), columns = category_names)
y_preds.head()

,medical_products,buildings,earthquake,request,storm,aid_related,infrastructure_related,transport,death,floods,...,water,shelter,weather_related,refugees,security,military,other_infrastructure,hospitals,food,child_alone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
y_test.shape, y_preds.shape

((6554, 36), (6554, 36))

In [14]:
def evaluate_mode(model, X_test, y_test, col_name):
    """ Evaluate model on test set,
        Predict results for each category.
        
    Args:
        model: trained model
        X_test: pandas.DataFrame for predict 
        y_test: pandas.DataFrame for labeled test set
        col_name: list for category names
        `
    Returns: 
        none.
    
    """
    
    # predict test df
    Y_pred = model.predict(X_test)
    tot_acc = 0
    tot_f1 = 0
    # print report 
    for i, cat in enumerate(col_name):    
        metrics =  classification_report(y_test[y_test.columns[i]], Y_pred[:,i])
        tot_acc += accuracy_score(y_test[y_test.columns[i]], Y_pred[:,i])
        tot_f1 += precision_recall_fscore_support(y_test[y_test.columns[i]], Y_pred[:,i], average = 'weighted')[2]
        print(cat, 'accuracy: {:.2f}'.format(accuracy_score(y_test[y_test.columns[i]], Y_pred[:,i])))
        print(metrics)
    print('total accuracy {:.2f}'.format(tot_acc/len(col_name)))
    print('total f1 {:.2f}'.format(tot_f1/len(col_name)))


In [15]:
evaluate_mode(pipeline, X_test, y_test, category_names)

medical_products accuracy: 0.95
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6212
           1       0.74      0.08      0.15       342

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.84      0.54      0.56      6554
weighted avg       0.94      0.95      0.93      6554

buildings accuracy: 0.95
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6205
           1       0.69      0.12      0.21       349

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.82      0.56      0.59      6554
weighted avg       0.94      0.95      0.93      6554

earthquake accuracy: 0.97
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5934
           1       0.90      0.71      0.80       620

   micro avg       0.97      0.97      0.97      6554
   macro avg       0.94      0.85      0.89  

/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


related accuracy: 0.81
              precision    recall  f1-score   support

           0       0.64      0.49      0.55      1519
           1       0.85      0.92      0.88      4988
           2       0.92      0.26      0.40        47

   micro avg       0.81      0.81      0.81      6554
   macro avg       0.80      0.55      0.61      6554
weighted avg       0.80      0.81      0.80      6554

other_weather accuracy: 0.95
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6206
           1       0.70      0.07      0.14       348

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.83      0.54      0.55      6554
weighted avg       0.94      0.95      0.93      6554

tools accuracy: 0.99
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6517
           1       0.00      0.00      0.00        37

   micro avg       0.99      0.99      0.99      6554
 

### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
parameters = {
        'tfidfvect__ngram_range': [(1, 1), (1, 2)],
        'tfidfvect__max_df': [0.75, 1.0],
        'clf__estimator__n_estimators': [10, 15],
        'clf__estimator__min_samples_split': [2, 5]
        }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv_model = cv.fit(X_train, y_train)


/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  sorted(inconsistent))
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  sorted(inconsistent))
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated 

/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  sorted(inconsistent))
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  sorted(inconsistent))
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  sorted(inconsistent))
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not

In [ ]:
evaluate_mode(cv_model, X_test, y_test, Y.columns)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
class TextLengthExtractor(BaseEstimator, TransformerMixin):
    '''
    Add the length of the text message as a feature to dataset
    
    The assumption is people who is in urgent disaster condition will prefer to use less words to express
    '''
    
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        return pd.DataFrame(X).applymap(len)



In [ ]:
pipeline_2 = Pipeline([
    ('features', FeatureUnion([       
        ('tfidfvect', TfidfVectorizer(tokenizer=tokenize)),
        ('text_total_length', TextLengthExtractor()) # Add a new feature
        ])),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42))) # Change the estimator to AdaBoost
    ])

In [ ]:
parameters_2 = {
        'features__tfidfvect__ngram_range': [(1, 1), (1, 2)],
        'features__tfidfvect__max_df': [0.75, 1.0],
        'features__tfidfvect__max_features': [2000, 5000],
        'features__transformer_weights': (
            {'tfidfvect': 1, 'text_total_length': 0.5},
            {'tfidfvect': 1, 'text_total_length': 1}
        ),
        'clf__estimator__n_estimators': [10, 15],
        'clf__estimator__learning_rate': [0.05, 0.1]
        }

cv_2 = GridSearchCV(pipeline_2, param_grid=parameters_2)

In [ ]:
cv_2_model = cv_2.fit(X_train, y_train)


In [ ]:
evaluate_mode(cv_2_model, X_test, y_test, Y.columns)


### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.